<a href="https://colab.research.google.com/github/monsoonfan/INF503_S2017_Project3/blob/master/RNGNBarazadeh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## helper


In [7]:
import numpy as np
import tensorflow as tf
import os
import shutil

def folder_creator(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(e)
def leaky_relu(x, alpha=0.2):
    return tf.maximum(x * alpha, x)

def conv2d(x, W):

    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_placeholder(shape, name=None):
    return tf.placeholder(tf.float32, shape, name = name)

def bias_placeholder(shape, name = None):
    return tf.placeholder(tf.float32, shape, name = name)

def weight_bias(hyper):

    global filter_size, fst_lyr_num_fltrs, scnd_lyr_num_fltrs, third_lyr_num_fltrs, forth_lyr_num_fltrs

    filter_size = hyper['filter_size']
    fst_lyr_num_fltrs = hyper['fst_lyr_num_fltrs']
    scnd_lyr_num_fltrs = hyper['scnd_lyr_num_fltrs']

    W_conv1 = weight_placeholder([filter_size, filter_size, 1, fst_lyr_num_fltrs], name = 'W_conv1' )
    b_conv1 = bias_placeholder([fst_lyr_num_fltrs], name = 'b_conv1')

    W_conv2 = weight_placeholder([filter_size, filter_size, fst_lyr_num_fltrs, scnd_lyr_num_fltrs], name = 'W_conv2')
    b_conv2 = bias_placeholder([scnd_lyr_num_fltrs], name = 'b_conv2')

    return W_conv1, b_conv1, W_conv2, b_conv2


def Generator(z, reuse = False):



    with tf.variable_scope("Generator", reuse=reuse):
        bs = tf.shape(z)[0]
        fc1 = tf.layers.dense(z, 1024)
        fc1 = leaky_relu(fc1)
        fc2 = tf.layers.dense(fc1, 7 * 7 * 128)
        fc2 = tf.reshape(fc2, tf.stack([bs, 7, 7, 128]))
        fc2 = leaky_relu(fc2)
        conv1 = tf.contrib.layers.conv2d_transpose(fc2, 64, [4, 4], [2, 2])
        conv1 = leaky_relu(conv1)
        conv2 = tf.contrib.layers.conv2d_transpose(conv1, 1, [4, 4], [2, 2], activation_fn=tf.sigmoid)
        conv2 = tf.reshape(conv2, tf.stack([bs, 784]))
    return conv2



def random_discriminator(u, rr, D1):


    u = tf.reshape(u, [-1,28,28,1])
    u = leaky_relu(conv2d(u, D1['W_conv1']) + D1['b_conv1'])
    u = max_pool_2x2(u)

    u = leaky_relu(conv2d(u, D1['W_conv2']) + D1['b_conv2'])
    u = max_pool_2x2(u)
    u = tf.reshape(u, [-1, 7*7*rr]) #tf.contrib.layers.flatten(u)#


    return u


def dist(a, b, bs):
    c = 0.0 * tf.norm(a[1,] - b[1,])
    for j in range(bs):
       vec = a[j,]
       m = bs
       matrix = tf.ones([m, 1]) * vec
       temp = tf.norm(matrix - b, axis = 1)

       c = c + tf.reduce_min(temp)

    return c


def data2img(data):
    shape = [28, 28, 1]
    return np.reshape(data, [data.shape[0]] + shape)

def grid_transform2(x, size):
    a = 8
    b = 16
    h, w, c = size[0], size[1], size[2]
    x = np.reshape(x, [a, b, h, w, c])
    x = np.transpose(x, [0, 2, 1, 3, 4])
    x = np.reshape(x, [a * h, b * w, c])
    if x.shape[2] == 1:
        x = np.squeeze(x, axis=2)
    return x

def grid_transform(x, size):

    a = 10
    b = int(x.shape[0]/a)
    h, w, c = size[0], size[1], size[2]
    x = np.reshape(x, [a, b, h, w, c])
    x = np.transpose(x, [0, 2, 1, 3, 4])
    x = np.reshape(x, [a * h, b * w, c])
    if x.shape[2] == 1:
        x = np.squeeze(x, axis=2)
    return x

def return_normal(shape):
    return np.float32(np.random.normal(loc=0.0, scale=0.1, size=shape))

def return_const(shape, c = .1):
    return np.float32(np.full(shape = shape, fill_value = c))

def return_unifrom(shape):
    return np.float32(np.random.uniform(-1.0, 1.0, shape))



## main

In [9]:
import matplotlib.pyplot as plt
import helper
#%tensorflow_version 2.x
#import tensorflow as tf
import numpy as np
from PIL import Image
import scipy.io as sio
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)




sample_dir = 'logs/mnist14'
sample_dir_2 = './icp_format_results14/'
sample_dir_sample = 'logs/sample'

z_dim = 100
b_size = 100
max_iter = 100000
step_size = 1e-3
iteration = 500
hyper_parm_filter = {
'filter_size' : 5, ##### Best is 16
'fst_lyr_num_fltrs' : 64,
'scnd_lyr_num_fltrs' : 64,
}
rr = hyper_parm_filter['scnd_lyr_num_fltrs']



helper.folder_creator(sample_dir)
helper.folder_creator(sample_dir_2)
helper.folder_creator(sample_dir_sample)




W_conv1_temp, b_conv1_temp, W_conv2_temp, b_conv2_temp = helper.weight_bias(hyper_parm_filter)
W_conv1 = tf.get_variable(dtype = tf.float32, name = 'W_conv1',
initializer=W_conv1_temp)
b_conv1 = tf.get_variable(dtype = tf.float32,  name = 'b_conv1',
initializer=b_conv1_temp)
W_conv2 = tf.get_variable(dtype = tf.float32, name = 'W_conv2',
initializer=W_conv2_temp)
b_conv2 = tf.get_variable(dtype = tf.float32,  name = 'b_conv2',
initializer=b_conv2_temp)



D1 = {'W_conv1':W_conv1,'b_conv1':b_conv1, 'W_conv2':W_conv2,'b_conv2':b_conv2}
init_new_vars_op = tf.initialize_variables([W_conv1, b_conv1, W_conv2, b_conv2])

x1 = tf.placeholder(tf.float32, [None, 784], name = 'x1')
z1 = tf.placeholder(tf.float32, [None, z_dim], name = 'z1')

gz1 = helper.Generator(z1)


x1_real_conv = helper.random_discriminator(x1,rr,D1 )


gz1_real_conv = helper.random_discriminator(gz1,rr, D1)




loss_1 = helper.dist(x1_real_conv , gz1_real_conv, b_size)
loss_2 = helper.dist( gz1_real_conv, x1_real_conv, b_size)
g_loss =  tf.cond(tf.less(loss_1, loss_2), lambda: loss_2, lambda: loss_1)

gen_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Generator")


step = tf.Variable(0, trainable=False)
rate = tf.train.exponential_decay(step_size, step, 3000, 0.96)


optimizer = tf.train.AdamOptimizer(learning_rate=rate, beta1=0.5, beta2=0.9, name = 'Adam1')
grads_and_vars = optimizer.compute_gradients(g_loss,  var_list=gen_params )
vars = [x[1] for x in grads_and_vars]
gradients = [x[0] for x in grads_and_vars]
gen_train_op = optimizer.apply_gradients(zip(gradients,vars), global_step = step )


#
def Create_feed_dict():
    filter_size = hyper_parm_filter['filter_size']
    fst_lyr_num_fltrs = hyper_parm_filter['fst_lyr_num_fltrs']
    scnd_lyr_num_fltrs = hyper_parm_filter['scnd_lyr_num_fltrs']
    feed = {}
    #
    feed[b_conv1_temp] = helper.return_const([fst_lyr_num_fltrs], c = 0.0)
    #
    feed[b_conv2_temp] = helper.return_const([scnd_lyr_num_fltrs], c = 0.0 )
    feed[W_conv1_temp] = helper.return_normal([filter_size, filter_size, 1, fst_lyr_num_fltrs])
    feed[W_conv2_temp] = helper.return_normal([filter_size, filter_size, fst_lyr_num_fltrs, scnd_lyr_num_fltrs])

    return feed

def Generate_samples_and_save(fixed_noise, iter = 0, counter = 1):
    fake_samples = sess.run(gz1, feed_dict={z1:fixed_noise})

    adict = {}
    adict['images'] = fake_samples
    c = counter
    sio.savemat(sample_dir_2+'{}.mat'.format(str(c).zfill(3)), adict)
    fake_samples = helper.data2img(fake_samples)
    sample_plot = fake_samples[np.random.randint(0, 10000, 128),:]


    fake_samples = helper.grid_transform(fake_samples, [28, 28, 1])
    fake_samples = np.squeeze(fake_samples)
    fake_samples = (255.99*fake_samples).astype('uint8')
    plt.imsave(sample_dir+'/samples_'+str(iter)+'.png', fake_samples)
    img = Image.open(sample_dir+'/samples_'+str(iter)+'.png').convert('LA')
    img.save(sample_dir+'/samples_'+str(iter)+'.png')
    sample_plot = helper.grid_transform2(sample_plot, [28, 28, 1])
    sample_plot = np.squeeze(sample_plot)
    sample_plot = (255.99 * sample_plot).astype('uint8')
    plt.imsave(sample_dir_sample + '/samples_' + str(iter) + '.png', sample_plot)
    img = Image.open(sample_dir_sample + '/samples_' + str(iter) + '.png').convert('LA')
    img.save(sample_dir_sample + '/samples_' + str(iter) + '.png')

    return fake_samples



fixed_noise = np.random.normal(0.0, 1.0, [10000, z_dim])

with tf.Session() as sess:

    feed2 = Create_feed_dict()
    sess.run(tf.global_variables_initializer(),feed_dict = feed2)
    iter = 0
    Plot_counter = 1
    Generate_samples_and_save(fixed_noise, iter = iter, counter = 0 )
    while iter < (max_iter):
        if iter % 10 == 0:
            print(iter)

        feed = {}
        data1 = mnist.train.next_batch(b_size)[0]
        code1 = helper.return_unifrom([b_size , z_dim])
        feed[x1] = data1
        feed[z1] = code1
        _= sess.run(gen_train_op, feed_dict = feed)

        iter = iter + 1
        feed2 = Create_feed_dict()
        sess.run(init_new_vars_op, feed_dict = feed2)

        if iter % 500 == 0:
            Generate_samples_and_save(fixed_noise, iter = iter, counter = Plot_counter )
            Plot_counter = Plot_counter  + 1



11493376/11490434 [==============================] - 0s 0us/step


AttributeError: ignored